In [1]:
# Import library
import pandas as pd 
import os 
import pandas.tseries.offsets as offsets 

In [2]:
# Get file path
file_list = os.listdir(path='./data')
file_list.remove('.DS_Store') # If Mac, it is neccecary to remove
print(file_list)

['Report - 2020-02-13T150639.682.xls.xlsx']


In [3]:
# Create dataframe(df)
# Each columns are written in used language
df = pd.DataFrame(index=[], columns=['ID', '活動量日時', '日乳量', '反芻注意', '合計反芻時間', '活動量', '発情の可能性',  '活動量注意'])

In [4]:
# View df
df

,ID,活動量日時,日乳量,反芻注意,合計反芻時間,活動量,発情の可能性,活動量注意


In [5]:
# Read data of columns of all .xlsx files
# Coution: Too much data cannot be read at once (memory overload)
for i in file_list:
    df1 = pd.read_excel('./data/{0}'.format(i), header=None)
    ID = df1.iat[2, 3]
    ID = str(ID)
    ID = ID[:4]
    df2 = pd.read_excel('./data/{0}'.format(i), header=8, usecols=[0,1,2,3,4,5,6,7], skiprows=[9,10,11]) # Skiprows contain html metadata
    df2.columns = ['ID', '活動量日時', '日乳量', '反芻注意', '合計反芻時間', '活動量', '発情の可能性',  '活動量注意']
    df2['ID'] = ID
    df = df.append(df2, ignore_index=True)

In [6]:
# View df
df

,ID,活動量日時,日乳量,反芻注意,合計反芻時間,活動量,発情の可能性,活動量注意
0,3090,2013-02-20 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,3090,2013-02-20 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,3090,2013-02-20 10:00:00,NaN,NaN,588.0,27.0,-14.0,NaN
3,3090,2013-02-20 08:00:00,NaN,NaN,568.0,39.0,-14.0,NaN
4,3090,2013-02-20 06:00:00,NaN,NaN,581.0,49.0,-12.0,NaN
...,...,...,...,...,...,...,...,...
4119,3090,2007-03-19 08:00:00,NaN,NaN,12.0,43.0,NaN,NaN
4120,3090,2007-03-19 06:00:00,NaN,NaN,8.0,46.0,NaN,NaN
4121,3090,2007-03-19 04:00:00,NaN,NaN,2.0,32.0,NaN,NaN
4122,3090,2007-03-19 02:00:00,NaN,NaN,0.0,23.0,NaN,NaN


In [7]:
# Convert date data type to string to correct date ordering
# At 2020, T4C was unable to write out the Japanese date markings correctly.
df['活動量日時'] = df['活動量日時'].astype(str)

In [8]:
# 確認
df['活動量日時']

0         2013-02-20 14:00:00
1         2013-02-20 12:00:00
2         2013-02-20 10:00:00
3         2013-02-20 08:00:00
4         2013-02-20 06:00:00
                 ...         
425380    2018-02-19 08:00:00
425381    2018-02-19 06:00:00
425382    2018-02-19 04:00:00
425383    2018-02-19 02:00:00
425384    2018-02-19 00:00:00
Name: 活動量日時, Length: 425385, dtype: object

In [9]:
# Check the numbers of rows
len(df)

425385

In [10]:
# Fixing the date sequence
for i in range(len(df)):
    date = df.at[i, '活動量日時']
    df.at[i, '活動量日時'] = "20" + date[8:10] + "-" + date[5:7] + "-" + date[2:4] + " " + date[11:]

In [11]:
#　Confirmation
df['活動量日時']

0         2020-02-13 14:00:00
1         2020-02-13 12:00:00
2         2020-02-13 10:00:00
3         2020-02-13 08:00:00
4         2020-02-13 06:00:00
                 ...         
425380    2019-02-18 08:00:00
425381    2019-02-18 06:00:00
425382    2019-02-18 04:00:00
425383    2019-02-18 02:00:00
425384    2019-02-18 00:00:00
Name: 活動量日時, Length: 425385, dtype: object

In [12]:
# Save df
df.to_excel('./data.xlsx')

In [13]:
# Only the row with milk volume (at 0:00) is extracted and stored in df3.
df3 = df[df.日乳量 > 0]

In [14]:
# Reworking the index of df3
df3.reset_index(drop=True, inplace=True)
df3

,ID,活動量日時,日乳量,反芻注意,合計反芻時間,活動量,発情の可能性,活動量注意
0,3107,2019-12-30 00:00:00,26.2,NaN,552.0,27.0,-7.0,NaN
1,3107,2019-12-29 00:00:00,26.8,x,376.0,35.0,0.0,NaN
2,3107,2019-12-28 00:00:00,25.6,NaN,514.0,36.0,5.0,NaN
3,3107,2019-12-27 00:00:00,27.7,NaN,540.0,44.0,13.0,NaN
4,3107,2019-12-26 00:00:00,28.2,NaN,420.0,39.0,-1.0,NaN
...,...,...,...,...,...,...,...,...
26389,3112,2019-02-22 00:00:00,35.8,NaN,486.0,31.0,-1.0,NaN
26390,3112,2019-02-21 00:00:00,35.7,NaN,514.0,27.0,-2.0,NaN
26391,3112,2019-02-20 00:00:00,35.8,NaN,505.0,33.0,13.0,NaN
26392,3112,2019-02-19 00:00:00,35.9,NaN,491.0,26.0,4.0,NaN


In [16]:
# Convert date data type from string to datetime
pd.to_datetime(df3['活動量日時'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

0       2019-12-30
1       2019-12-29
2       2019-12-28
3       2019-12-27
4       2019-12-26
           ...    
26389   2019-02-22
26390   2019-02-21
26391   2019-02-20
26392   2019-02-19
26393   2019-02-18
Name: 活動量日時, Length: 26394, dtype: datetime64[ns]

In [17]:
# View df3
df3

,ID,活動量日時,日乳量,反芻注意,合計反芻時間,活動量,発情の可能性,活動量注意
0,3107,2019-12-30 00:00:00,26.2,NaN,552.0,27.0,-7.0,NaN
1,3107,2019-12-29 00:00:00,26.8,x,376.0,35.0,0.0,NaN
2,3107,2019-12-28 00:00:00,25.6,NaN,514.0,36.0,5.0,NaN
3,3107,2019-12-27 00:00:00,27.7,NaN,540.0,44.0,13.0,NaN
4,3107,2019-12-26 00:00:00,28.2,NaN,420.0,39.0,-1.0,NaN
...,...,...,...,...,...,...,...,...
26389,3112,2019-02-22 00:00:00,35.8,NaN,486.0,31.0,-1.0,NaN
26390,3112,2019-02-21 00:00:00,35.7,NaN,514.0,27.0,-2.0,NaN
26391,3112,2019-02-20 00:00:00,35.8,NaN,505.0,33.0,13.0,NaN
26392,3112,2019-02-19 00:00:00,35.9,NaN,491.0,26.0,4.0,NaN


In [18]:
# Select the date range to read
start_date = pd.to_datetime('2018-10-01 00:00:00')# Start
end_date = pd.to_datetime('2020-02-13 00:00:00') # End
daterange = end_date - start_date + offsets.Day(1) 
daterange_int = daterange.days # Numbers of days to read

In [19]:
daterange_int

501

In [20]:
# Create df4 (for inserting 日乳量 (Milk yield  kg/day))
df4 = pd.DataFrame(index=[], columns=['ID'])
df4.set_index('ID')

""
ID


In [21]:
df3.drop_duplicates(inplace=True)

/Users/paw/pandas/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
# Insert milk yield for each cow by date into df4
for i in range(daterange_int):
    date = start_date + offsets.Day(i)
    date_str = date.strftime("%Y-%m-%d %H:%M:%S")
    df5 = df3.query('活動量日時 in [@date_str]')
    df6 = df5.loc[:, ['ID', '日乳量']]
    df7 = df6.set_index('ID')
    df8 = df7.rename(columns={'日乳量': '{}'.format(date_str)})
    df4 = pd.concat([df4, df8], axis=1, sort=True)

In [23]:
df4

,ID,2018-10-01 00:00:00,2018-10-02 00:00:00,2018-10-03 00:00:00,2018-10-04 00:00:00,2018-10-05 00:00:00,2018-10-06 00:00:00,2018-10-07 00:00:00,2018-10-08 00:00:00,2018-10-09 00:00:00,...,2020-02-04 00:00:00,2020-02-05 00:00:00,2020-02-06 00:00:00,2020-02-07 00:00:00,2020-02-08 00:00:00,2020-02-09 00:00:00,2020-02-10 00:00:00,2020-02-11 00:00:00,2020-02-12 00:00:00,2020-02-13 00:00:00
2606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.5,34.7,34.4,34.8,34.4,33.7,32.5,31.0,33.3,32.6
2619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.2,56.3,50.1,65.0,63.9,64.0,64.8,62.9,65.0,64.4
2622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.6,34.0,34.4,34.7,34.6,34.9,32.9,31.9,32.7,34.6
2626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.2,54.1,55.0,57.4,55.5,55.0,55.0,53.7,53.5,53.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.7,31.8,31.6,32.4,30.7,29.4,31.6,31.8,30.6,31.4
3144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.6,22.2,27.5,26.8,27.5,28.2,28.4,23.5,19.5,17.3
3145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.1,30.5,28.0,29.2,28.3,27.9,26.5,28.0,29.4,28.6


In [24]:
# Drop the row 'ID'
df9 = df4.drop('ID', axis=1) 

In [25]:
df9

,2018-10-01 00:00:00,2018-10-02 00:00:00,2018-10-03 00:00:00,2018-10-04 00:00:00,2018-10-05 00:00:00,2018-10-06 00:00:00,2018-10-07 00:00:00,2018-10-08 00:00:00,2018-10-09 00:00:00,2018-10-10 00:00:00,...,2020-02-04 00:00:00,2020-02-05 00:00:00,2020-02-06 00:00:00,2020-02-07 00:00:00,2020-02-08 00:00:00,2020-02-09 00:00:00,2020-02-10 00:00:00,2020-02-11 00:00:00,2020-02-12 00:00:00,2020-02-13 00:00:00
2606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.5,34.7,34.4,34.8,34.4,33.7,32.5,31.0,33.3,32.6
2619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.2,56.3,50.1,65.0,63.9,64.0,64.8,62.9,65.0,64.4
2622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.6,34.0,34.4,34.7,34.6,34.9,32.9,31.9,32.7,34.6
2626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.2,54.1,55.0,57.4,55.5,55.0,55.0,53.7,53.5,53.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.7,31.8,31.6,32.4,30.7,29.4,31.6,31.8,30.6,31.4
3144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.6,22.2,27.5,26.8,27.5,28.2,28.4,23.5,19.5,17.3
3145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.1,30.5,28.0,29.2,28.3,27.9,26.5,28.0,29.4,28.6


In [26]:
# Save  df9 to xlsx format
df9.to_excel('./nyuryo.xlsx')

In [27]:
# Create df4 (for inserting 反芻時間 (Rumination time  min/day))
df10 = pd.DataFrame(index=[], columns=['ID'])
df10.set_index('ID')

""
ID


In [28]:
# # Insert rumination time for each cow by date into df10
for i in range(daterange_int):
    date2 = start_date + offsets.Day(i) # 抽出日の初期化
    date2_str = date2.strftime("%Y-%m-%d %H:%M:%S") # 抽出日を文字列変換
    df11 = df3.query('活動量日時 in [@date2_str]') # 抽出日のデータ抽出
    df12 = df11.loc[:, ['ID', '合計反芻時間']] # IDと合計反芻時間を抽出
    df13 = df12.set_index('ID') # IDをインデックスにはめ込む
    df14 = df13.rename(columns={'合計反芻時間': '{}'.format(date2_str)}) # 合計反芻時間のラベルを日付に変換
    df10 = pd.concat([df10, df14], axis=1, sort=True) # df10に埋め込んでいく（indexはpandasが勝手にソートする）

In [29]:
# View df10
df10

,ID,2018-10-01 00:00:00,2018-10-02 00:00:00,2018-10-03 00:00:00,2018-10-04 00:00:00,2018-10-05 00:00:00,2018-10-06 00:00:00,2018-10-07 00:00:00,2018-10-08 00:00:00,2018-10-09 00:00:00,...,2020-02-04 00:00:00,2020-02-05 00:00:00,2020-02-06 00:00:00,2020-02-07 00:00:00,2020-02-08 00:00:00,2020-02-09 00:00:00,2020-02-10 00:00:00,2020-02-11 00:00:00,2020-02-12 00:00:00,2020-02-13 00:00:00
2606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,492.0,527.0,577.0,526.0,582.0,504.0,539.0,450.0,578.0,549.0
2619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,535.0,459.0,476.0,524.0,501.0,461.0,511.0,564.0,475.0,525.0
2622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,699.0,696.0,651.0,649.0,640.0,703.0,763.0,710.0,619.0,700.0
2626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,477.0,437.0,426.0,445.0,436.0,421.0,498.0,450.0,411.0,448.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,466.0,443.0,459.0,444.0,400.0,429.0,393.0,439.0,420.0,464.0
3144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,547.0,521.0,553.0,450.0,459.0,521.0,520.0,509.0,508.0,489.0
3145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,526.0,602.0,555.0,611.0,500.0,535.0,575.0,549.0,572.0,582.0


In [30]:
# Remove the row 'ID'
df15 = df15.drop('ID', axis=1)

In [31]:
df15

,2018-10-01 00:00:00,2018-10-02 00:00:00,2018-10-03 00:00:00,2018-10-04 00:00:00,2018-10-05 00:00:00,2018-10-06 00:00:00,2018-10-07 00:00:00,2018-10-08 00:00:00,2018-10-09 00:00:00,2018-10-10 00:00:00,...,2020-02-04 00:00:00,2020-02-05 00:00:00,2020-02-06 00:00:00,2020-02-07 00:00:00,2020-02-08 00:00:00,2020-02-09 00:00:00,2020-02-10 00:00:00,2020-02-11 00:00:00,2020-02-12 00:00:00,2020-02-13 00:00:00
2606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,492.0,527.0,577.0,526.0,582.0,504.0,539.0,450.0,578.0,549.0
2619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,535.0,459.0,476.0,524.0,501.0,461.0,511.0,564.0,475.0,525.0
2622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,699.0,696.0,651.0,649.0,640.0,703.0,763.0,710.0,619.0,700.0
2626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,477.0,437.0,426.0,445.0,436.0,421.0,498.0,450.0,411.0,448.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,466.0,443.0,459.0,444.0,400.0,429.0,393.0,439.0,420.0,464.0
3144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,547.0,521.0,553.0,450.0,459.0,521.0,520.0,509.0,508.0,489.0
3145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,526.0,602.0,555.0,611.0,500.0,535.0,575.0,549.0,572.0,582.0


In [32]:
# Save df15 to xlsx format
df15.to_excel('./hansu.xlsx')